In [1]:
import cv2
import numpy as np
import os
import time
import random
import matplotlib.pyplot as plt
from uuid import getnode as get_mac
mac = get_mac()

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred,{'projectId': "facemaskdetector-987ed",})
db = firestore.client()

device_ref = db.collection('devices')

query_ref = device_ref.where('id','==', mac)

if len(query_ref.get())==0:
    doc_ref = db.collection('devices').document(str(random.random()))
    doc_ref.set({'mac':mac,'count': 0},merge=True)

In [3]:
def setPeople(i):
    col_ref = db.collection('devices')
    query_ref = col_ref.where('mac','==', mac).get()
    field_updates = {'count':i}
    for item in query_ref:
        doc = col_ref.document(item.id)
        doc.update(field_updates)

In [4]:
net = cv2.dnn.readNetFromDarknet("C:\\Users\\mihir\\OneDrive\\Desktop\\Study\\DesignEngineering\\FaceMask\\darknet\\cfg\\yolov3-tiny_custom.cfg","yolov3-tiny_custom_6000.weights")

In [5]:
classes = ['mask_weared_incorrect','with_mask','without_mask']

In [6]:
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4',fourcc,20.0,(1280,720))

t = round(time.time()*1000)
while 1:
    _, img = cap.read()
    img = cv2.resize(img,(416,416))
    hight,width,_ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255,(416,416),(0,0,0),swapRB = True,crop= False)

    net.setInput(blob)

    output_layers_name = net.getUnconnectedOutLayersNames()

    layerOutputs = net.forward(output_layers_name)

    boxes =[]
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * hight)
                w = int(detection[2] * width)
                h = int(detection[3]* hight)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,.8,.4)
    font = cv2.FONT_HERSHEY_PLAIN
    
    temp = round(time.time()*1000)
    if temp-t >= 5000:
        setPeople(len(indexes))
        t=round(time.time()*1000)
    
    if  len(indexes)>0:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = [255,255,255]
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,label + " " + confidence, (x,y+h),font,2,color,2)
        cv2.putText(img,"Count : " + str(len(indexes)), (100,100),font,2,[255,0,0],2)  
            
    out.write(img)
    cv2.imshow('img',img)
    
   
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
count = 0

cap = cv2.VideoCapture(0)

while 1:
    _, img = cap.read()
    img = cv2.resize(img,(416,416))
    
    if cv2.waitKey(1) == ord('w'):
        cv2.imwrite(os.path.join("C:\\Users\\mihir\\OneDrive\\Desktop\\Study\\DesignEngineering\\FaceMask\\new_data", '%d.png') % count, img)
        count += 1
    
    cv2.imshow('img',img)
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
import smtplib, ssl

smtp_server = "smtp.gmail.com"
port = 587  # For starttls
sender_email = "wisemanmihir@gmail.com"
password ="m!h!r1245"

# Create a secure SSL context
context = ssl.create_default_context()

# Try to log in to server and send email
try:
    server = smtplib.SMTP(smtp_server,port)
    server.ehlo() # Can be omitted
    server.starttls(context=context) # Secure the connection
    server.ehlo() # Can be omitted
    server.login(sender_email, password)
    server.sendmail(sender_email, "mihirchauhan756@gmail.com", "Hello")

except Exception as e:
    # Print any error messages to stdout
    print(e)
finally:
    server.quit() 